In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import vcf

from IPython.display import display
from ipywidgets import FloatProgress

%matplotlib inline

DIR = r'c://downloads'

plt.style.use('ggplot')

In [2]:
brca1_variants = pd.read_csv(os.path.join(DIR, 'brca1_variants.csv'))
display(brca1_variants)

,HGVS Variant,Pathogenicity,Pathogenic,Start,End,Reference Seq,Variant Seq,Reference Seq Complement,Variant Seq Complement
0,c.8T>G,5,True,41276105,41276105,A,C,T,G
1,c.32_33insC,5,True,41276080,41276081,TA,TGA,TA,TCA
2,c.34C>T,5,True,41276079,41276079,G,A,C,T
3,c.38_39delATinsGGG,5,True,41276074,41276075,AT,CCC,AT,GGG
4,c.53T>C,4,True,41276060,41276060,A,G,T,C
5,c.55C>T,5,True,41276058,41276058,G,A,C,T
6,c.61delA,5,True,41276052,41276052,T,NaN,A,NaN
7,c.62dupT,5,True,41276051,41276051,A,AA,T,TT
8,c.64_65delTT,5,True,41276048,41276049,AA,NaN,TT,NaN
9,c.65T>C,5,True,41276048,41276048,A,G,T,C


In [3]:
# We hold a cache of all pathogenic variants in a set (for fast lookup). Each variant is represented as a tuple of:
# (location (0-based index; int), ref seq (str), alt seq (str)).

pathogenic_variants = set(brca1_variants[brca1_variants['Pathogenic']].apply(lambda record: \
        (record['Start'], record['Reference Seq'], record['Variant Seq']), axis = 1))

print(len(pathogenic_variants))
print(list(pathogenic_variants)[:5])

1085
[(41219668, 'G', nan), (41245340, 'T', 'G'), (41244711, 'TA', nan), (41209079, 'G', 'GG'), (41244033, 'C', 'A')]


In [4]:
VCF_FILE_NAME = 'ALL.chr17.integrated_phase1_v3.20101123.snps_indels_svs.genotypes.41100000-41300000.vcf.gz'

for variant in vcf.Reader(filename = os.path.join(DIR, VCF_FILE_NAME), compressed = True):
    for allele_index, alt in enumerate(variant.ALT):
        
        # variant_key will have the same representation as the elements in the lookup set of pathogenic variants.
        # Note that we convert the position from 1-based to 0-based index, and that an empty sequence is represented as np.nan.
        variant_key = (variant.POS - 1, str(variant.REF), np.nan if len(alt) == 0 else str(alt))
        
        if variant_key in pathogenic_variants:
            for call in variant.samples:
                # call.gt_alleles gives the genotypes of the current sample as numbers. 0 represents the ref allele, 1 represents
                # the first alternative allele, and so on. On the other hand, allele_index starts counting the first alternative
                # allele as 0. Therefore, we check whether (allele_index + 1) is in the list of allele numbers of the genotype
                # of the current sample. 
                if (allele_index + 1) in map(int, call.gt_alleles):
                    print('Sample %s has the variant %s --> %s at position %d (%s)' % \
                            (call.sample, variant.REF, alt, variant.POS, variant.ID))

Sample HG00403 has the variant T --> A at position 41201105 (rs80358092)
Sample HG00537 has the variant T --> A at position 41201105 (rs80358092)
Sample HG00626 has the variant T --> A at position 41201105 (rs80358092)
Sample NA18528 has the variant T --> A at position 41201105 (rs80358092)
Sample NA18570 has the variant T --> A at position 41201105 (rs80358092)
Sample NA18613 has the variant T --> A at position 41201105 (rs80358092)
Sample NA18641 has the variant T --> A at position 41201105 (rs80358092)
Sample NA19780 has the variant A --> C at position 41243840 (rs28897687)
Sample NA19380 has the variant C --> G at position 41243948 (rs56214134)
Sample NA19382 has the variant C --> G at position 41243948 (rs56214134)
